# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("WeatherPy.csv")
cities.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Mukhen,48.10,136.10,67.50,80,3,1.10,RU
1,Atar,20.52,-13.05,98.58,15,2,5.93,MR
2,Nikolskoye,59.70,30.79,59.00,77,40,13.42,RU
3,Leona Vicario,20.97,-87.19,89.60,66,20,13.87,MX
4,Chokurdakh,70.63,147.92,52.95,40,34,9.64,RU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Latitude","Longitude"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df = cities.loc[(cities["Max Temp"] > 73) & (cities["Max Temp"] < 85) & (cities["Cloudiness"]==0), :]
hotel_df = hotel_df.dropna(how='any')
hotel_df.reset_index(inplace=True)
del hotel_df["index"]
hotel_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Quartucciu,39.25,9.18,80.01,57,0,9.17,IT
1,Darnah,32.77,22.64,75.25,69,0,16.96,LY
2,Misratah,32.38,15.09,73.15,77,0,3.27,LY
3,Sumenep,-7.02,113.87,78.40,81,0,7.99,ID
4,Sakakah,29.97,40.21,80.60,13,0,6.93,SA


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotellist =[]

for x in range(len(hotel_df)):
    lat = hotel_df.loc[x]['Latitude']
    lng = hotel_df.loc[x]['Longitude']
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types": "hotel",
        "key": g_key,
        }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request=requests.get(base_url, params=params)
    json = request.json()
    try:
        hotellist.append(json["results"][0]["name"])
    except:
        hotellist.append(" ")
hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,Quartucciu,39.25,9.18,80.01,57,0,9.17,IT,Cagliari
1,Darnah,32.77,22.64,75.25,69,0,16.96,LY,Derna
2,Misratah,32.38,15.09,73.15,77,0,3.27,LY,Misrata
3,Sumenep,-7.02,113.87,78.40,81,0,7.99,ID,Sumenep
4,Sakakah,29.97,40.21,80.60,13,0,6.93,SA,Sakaka


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))